# Storytelling with COVID-19

## Set-up

In [438]:
# Importing Libraries

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

import matplotlib.style as style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 150

import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px

from plotly.offline import iplot, init_notebook_mode
import plotly.io as pio
import cufflinks as cf
cf.go_offline(connected=True)
init_notebook_mode(connected=True)

cf.go_offline()
pio.renderers.default = 'colab'  # For Colab

import folium

In [426]:
#Importing data and renaming columns

df = pd.read_csv("/content/covid_19_data.csv", parse_dates=['Last Update'])
df.rename(columns={'Country/Region':'Country'}, inplace=True)
df['Country'] = df['Country'].replace('US', 'United States')
df['Country'] = df['Country'].replace('UK', 'United Kingdom')
df['Country'] = df['Country'].replace('Congo (Brazzaville)', 'DR Congo')
df['Country'] = df['Country'].replace('Burma', 'Myanmar')
df = df.drop(columns = ['SNo', "Last Update"])

df_conf = pd.read_csv("/content/time_series_covid_19_confirmed.csv")
df_conf.rename(columns={'Country/Region':'Country'}, inplace=True)

df_death = pd.read_csv("/content/time_series_covid_19_deaths.csv")
df_death.rename(columns={'Country/Region':'Country'}, inplace=True)

countries = pd.read_csv("/content/countries_by_population_2019.csv")
countries.rename(columns={'name':'Country'}, inplace=True)

countries_iso = pd.read_csv("/content/country_codes_2020.csv")
countries_iso.rename(columns={'name':'Country'}, inplace=True)
countries_iso.rename(columns={'cca3':'ISO'}, inplace=True)

canada_covid = pd.read_csv('/content/covid19-download.csv')

## Time Series Plots

In [432]:
ts_country = df_conf.drop(columns = ['Lat', 'Long', 'Province/State'])
ts_country = ts_country.groupby(['Country']).sum()

# Countries with most cases on last date in dataframe
ts_country = ts_country.sort_values(by = ts_country.columns[-1], ascending = False).head(10)

ts_country.transpose().iplot(title='Time Series of Countries With The Most Confirmed Cases', dimensions=(1000, 600), theme='white')


In [433]:
ts_country = df_death.drop(columns = ['Lat', 'Long', 'Province/State'])
ts_country = ts_country.groupby(['Country']).sum()

# Countries with most cases on last date in dataframe
ts_country = ts_country.sort_values(by = ts_country.columns[-1], ascending = False).head(10)

ts_country.transpose().iplot(title = 'Time series of Countries with the most COVID-related Deaths', dimensions = (1000, 600), theme = 'white')

## Geographical World Maps

In [429]:
# Aggregating confirmed and death data
df_aggregated = df.groupby('Country').agg({'Confirmed': 'sum', 'Deaths': 'sum'}).reset_index()

# Merging data w country population data
df_country = df_aggregated.merge(countries[['Country', 'pop2019']], left_on='Country', right_on='Country', how='left')

# ISO codes from countries_iso
df_country = df_country.merge(countries_iso[['Country', 'ISO']], on='Country', how='left')

# Cases by country
df_country['Confirmed Cases'] = df_country['Confirmed'] / (df_country['pop2019'] / 1_000)

# Deaths by country
df_country['Confirmed Deaths'] = df_country['Deaths'] / (df_country['pop2019'] / 1_000)

In [434]:
fig = px.choropleth(df_country, locations="ISO",
                    color="Confirmed Cases",
                    hover_name="Country",
                    color_continuous_scale=px.colors.sequential.YlOrRd)

layout = go.Layout(
    title=go.layout.Title(
        text="",
        x=0.5
    ),
    font=dict(size=14),
    width=1000,
    height=600,
    margin=dict(l=0, r=0, b=0, t=30)
)

fig.update_layout(layout)

fig.show()


In [435]:
fig = px.choropleth(df_country, locations="ISO",
                    color="Confirmed Deaths",
                    hover_name="Country",
                    color_continuous_scale=px.colors.sequential.YlOrRd)

layout = go.Layout(
    title=go.layout.Title(
        text="COVID-19 Confirmed Deaths by Country",
        x=0.5
    ),
    font=dict(size=14),
    width = 1000,
    height = 600,
    margin=dict(l=0,r=0,b=0,t=30)
)

fig.update_layout(layout)

fig.show()